In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import warnings
from datetime import *
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
#원하는 기간의 날짜를 리스트로 반환하는 함수
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y.%m.%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates


def make_soup(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup




def last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
    '''
    max_count_per_day = 크롤링할 일별 최대 뉴스의 개수
    
    
    네이버 뉴스페이지 구성

    시작페이지=1 step=10 으로 한 페이지당 10개의 뉴스를 노출시킨다.
    최대 노출 뉴스의 수는 400page 4000개의 뉴스를 노출시킨다. (그 이상은 불가능 필요하다면 검색 조건을 더 세부적으로 해야함)
    
    만약 검색한 키워드의 뉴스 총 노출수가 300개dlsep 100page를 url에 넣으면 알아서 300page로 들어가게된다

    따라서 마지막 페이지를 알아내려면 url에 page=4000을 넣으면 마지막 페이지로 이동할 수 있고
    그 soup에서 정규표현식을 통해 마지막 페이지의 숫자를 추출한다.



    날짜별로 반복:
        해당 날짜의 마지막 페이지 추출

        날짜, 마지막페이지번호, step 을 list에 추가

    return list
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #시작일 마감일 사이의 날짜들을 담은 리스트를 dates에 넣고 일 단위로 크롤링
    dates = date_range(startdate,enddate)
    
    date_lastpage_step = []

    for date in dates:  
        if max_count_per_day == -1:  
            url_ = url.format(search_keyword = search_keyword,
                            date = date,
                            date_ = date.replace('.',''),
                            page=4000)    # 네이버 뉴스페이지의 최대 노출 기사페이지 4000
            
            soup = make_soup(url_).select('a.btn') 
            
            try:    #검색 결과가 없는 경우 에러발생, 예외처리
                botton_count = int(re.findall('[0-9]+',str(soup[-1]).split(' ')[-1])[0]) * 10 - 9
                    
            except:
                print(date,'해당 일에 검색 결과가 존재하지 않습니다.')

        else:
            botton_count = max_count_per_day

        #페이지가 2이상 존재하지 않는 경우 step을 1로 설정
        step = 10 if botton_count > 10 else 1
        date_lastpage_step.append((date,botton_count,step))
    
    return date_lastpage_step



#start enddate default로 today()수정예정
def naver_news_url_crawling(search_keyword,max_count_per_day=-1,startdate='20220101',enddate='20220101'):
    '''
    startdate, enddate는 %Y%m%d 포맷으로 'YYYYMMDD' 형식
    max_count_per_day 하루 당 크롤링 할 최대 뉴스의 수 dafult시 (400,총 뉴스의 수) 중 작은값
    
    네이버 뉴스 웹은 page를 넘길 때 마다  (1, 11, 21, 31)순서로 바뀌고
    최대 page=4000 까지 지원한다. 페이지가 400이하일 때 page=4000을 넣으면 마지막 페이지가 나온다.
    매일 page=4000 url을 크롤링해서 마지막 페이지의 페이지버튼의 숫자를 통해 마지막 페이지를 크롤링하고
    for i in range(1, 총페이지수*10 -9, 10)으로 각 페이지마다 제목과 url 크롤링해서
    데이터프레임에 담아서 리턴 할 계획
    '''
    
    #검색하는 키워드의 띄어쓰기를 url상에 +으로 연결
    search_keyword = search_keyword.replace(' ','+')
    
    #네이버 뉴스 탭 url    
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #빈 데이터프레임을 만들고 크롤링한 데이터를 1일 단위로 concat해서 반환 할 계획
    result = pd.DataFrame(columns=['date','title','url'])
    
    

                                #last_botton_number 파라미터는 클래스 선언하면서 추후에 수정
                                #max_count_per_day -1일 경우 최대치로 크롤링 해야하는데 수정해야함
    for date, botton_count, step in last_botton_number(startdate,enddate,url,search_keyword,max_count_per_day):
        for page in range(1,botton_count+1,step): 
            url_ = url.format(search_keyword = search_keyword,
                              date = date,
                              date_= date.replace('.',''),
                              page = page)

            soup = make_soup(url_).select('a.news_tit')

            #한 페이지에 최대 노출 기사수 10개 1일 단위로 크롤링을 하니
            a_page_ulr = [html['href'] for html in soup[:min(10,max_count_per_day)]]
            a_page_title = [html['title'] for html in soup[:min(10,max_count_per_day)]]
            a_page_date = [date] * len(a_page_title)

            result= pd.concat([result , pd.DataFrame({'date':a_page_date, 'title':a_page_title, 'url':a_page_ulr})])
    
    # url에서 신문사의 정보를 유추할 수 있는 부분을 추출
    result['news_agency'] = result['url'].apply(lambda x : x.split('/')[2])
    
    return result




def main_text_crawling():
    
    
    '''
    각 신문사의 구조가 달라서 본문을 크롤링하는데에 제한이된다.
    발행수 상위 10개의 신문사의 데이터만 사용해서 본문을 크롤링하겠다.
    '''
    return None

In [8]:
df = naver_news_url_crawling(search_keyword='치킨',max_count_per_day=3,
                             startdate='20200101',enddate='20200110')
df

,date,title,url,news_agency
0,2020.01.01,"""치킨 시켜줘""…배달 앱 진화하며 음성만으로 음식 주문",http://yna.kr/AKR20191231039500030?did=1195m,yna.kr
1,2020.01.01,"""김정은, 트럼프 양보 얻어내려 압박 증대…위험한 치킨게임""",https://news.sbs.co.kr/news/endPage.do?news_id...,news.sbs.co.kr
2,2020.01.01,"평화당 ""北, 한반도 전쟁위험 몰아넣을 치킨게임 해선 안돼""",http://news1.kr/articles/?3806567,news1.kr
0,2020.01.01,"""김정은, 트럼프 양보 얻어내려 압박 증대…위험한 치킨게임""",http://yna.kr/AKR20200101039300071?did=1195m,yna.kr
1,2020.01.01,"평화당 ""北, 한반도 전쟁위험 몰아넣을 치킨게임 해선 안돼""",http://news1.kr/articles/?3806567,news1.kr
...,...,...,...,...
1,2020.01.10,"매운맛 치킨, '봄'에 가장 인기인 이유",http://www.sisaweek.com/news/articleView.html?...,www.sisaweek.com
2,2020.01.10,"""치킨부터 빙수까지""…밀레니얼 옷 입고 변신하는 ‘국민 메뉴’",http://www.ekn.kr/news/article.html?no=474922,www.ekn.kr
0,2020.01.10,"매운맛 치킨, '봄'에 가장 인기인 이유",http://www.sisaweek.com/news/articleView.html?...,www.sisaweek.com
1,2020.01.10,"""치킨부터 빙수까지""…밀레니얼 옷 입고 변신하는 ‘국민 메뉴’",http://www.ekn.kr/news/article.html?no=474922,www.ekn.kr


In [7]:
df['url']

0         http://yna.kr/AKR20191231039500030?did=1195m
0         http://yna.kr/AKR20200101039351071?did=1195m
0    http://news.mk.co.kr/newsRead.php?no=9079&year...
0    http://news.heraldcorp.com/view.php?ud=2020010...
0    http://moneys.mt.co.kr/news/mwView.php?no=2019...
0    http://www.queen.co.kr/news/articleView.html?i...
0         http://yna.kr/AKR20200107074500030?did=1195m
0    http://www.newsis.com/view/?id=NISX20200108_00...
0               http://kpenews.com/View.aspx?No=648360
0                    http://news1.kr/articles/?3813494
0            http://www.tenasia.co.kr/archives/1893346
0    http://moneys.mt.co.kr/news/mwView.php?no=2020...
0    http://www.queen.co.kr/news/articleView.html?i...
0    http://news.mt.co.kr/mtview.php?no=20200114100...
0    http://www.edaily.co.kr/news/newspath.asp?news...
0    https://news.chosun.com/site/data/html_dir/202...
0    http://sports.khan.co.kr/news/sk_index.html?ar...
0    http://www.leaders.kr/news/articleView.html?id...
0    http:

In [8]:
news_agency_list=['www.news1.kr',
 'www.newsis.com',
 'yna.kr',
 'news.kbs.co.kr',
 'www.edaily.co.kr',
 'www.shinailbo.co.kr',
 'www.fnnews.com',
 'news.mt.co.kr',
 'view.asiae.co.kr',
 'www.viva100.com']

df_ = df[df.news_agency.isin(news_agency_list)]
df_

,date,title,url,news_agency
0,2020.01.01,"""치킨 시켜줘""…배달 앱 진화하며 음성만으로 음식 주문",http://yna.kr/AKR20191231039500030?did=1195m,yna.kr
0,2020.01.02,"""김정은, 트럼프 양보 얻어내려 압박 증대…위험한 치킨게임""(종합)",http://yna.kr/AKR20200101039351071?did=1195m,yna.kr
0,2020.01.07,"bhc치킨 ""매운맛 치킨, 봄에 가장 많이 주문""",http://yna.kr/AKR20200107074500030?did=1195m,yna.kr
0,2020.01.08,"""1월1일 가장 많이 시켜먹은 음식은 치킨""",http://www.newsis.com/view/?id=NISX20200108_00...,www.newsis.com
0,2020.01.14,교촌치킨 사장님은 하루 110마리씩 튀겼다,http://news.mt.co.kr/mtview.php?no=20200114100...,news.mt.co.kr
0,2020.01.15,'골목식당' 치킨 메뉴 첫 등장 '백종원 평가는?',http://www.edaily.co.kr/news/newspath.asp?news...,www.edaily.co.kr
0,2020.01.20,"윤홍근 BBQ 회장, 10년만에 치킨대학 수료식 참석",http://www.fnnews.com/news/202001201557246724,www.fnnews.com


In [9]:
a = df.groupby('news_agency').count()['date'].sort_values(ascending=False)[:10]
a.index.tolist()

['yna.kr',
 'moneys.mt.co.kr',
 'www.queen.co.kr',
 'kpenews.com',
 'news.chosun.com',
 'news.heraldcorp.com',
 'news.mk.co.kr',
 'news.mt.co.kr',
 'news.wowtv.co.kr',
 'news1.kr']

In [ ]:
['www.news1.kr',
 'www.newsis.com',
 'yna.kr',
 'news.kbs.co.kr',
 'www.edaily.co.kr',
 'www.shinailbo.co.kr',
 'www.fnnews.com',
 'news.mt.co.kr',
 'view.asiae.co.kr',
 'www.viva100.com']

In [ ]:
a = 'http://moneys.mt.co.kr/news/mwView.php?no=2022010110548022148'
response = requests.get(a)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
b = str(soup.select('#_article div'))
def main_text_crawling(df):
    return None

"[<>핫한 SNS 치킨맛집 '이유치킨'이 최근 서울 방이점을 새롭게 오픈했다. 이유치킨의 가장 큰 장점은 독특한 메뉴 가성비다. 치킨을 신선한 기름에 튀기고 오븐에 굽는 두 번의 조리 과정으로 치킨의 풍미를 살렸다.\xa0</>, <></></>, <>육즙을 살린 베이크 치킨과 바삭한 식감의 크리스피 치킨으로 나눠 소비자 선택의 폭을 넓혔다. 가성비는 물론 인테리어와 패키지 디자인도 더욱 고급스럽게 신경썼다. 공격적인 오픈 이벤트와 맛과 가격을 모두 잡은 ‘갓성비’ 메뉴로 소비자 만족도를 더욱 높이고 있다.</>, <></></>, <>이유치킨의 대표적인 시그니처 메뉴인 ‘파스타치요’는 ‘파스타와 치킨이 만난 요리’라는 뜻의 파스타치요는 치킨과 치즈오븐 파스타를 합리적인 가격에 제공하고 있다. 이외에도 블랙갈릭, 메이플허니, 홍차오, 청양마요, 김치킹 등 다채로운 치킨과 사이드를 구비하고 있다.</>, <></></>, <>한편, 치킨 프랜차이즈 이유치킨은 브랜드변경(업종변경)시 창업자금을 지원해주고 있다.</>]"

In [ ]:
df.news_agency == 'moneys.mt.co.kr'

0    True 
1    False
2    False
3    False
4    False
     ...  
5    False
6    False
7    False
8    False
9    False
Name: news_agency, Length: 204, dtype: bool